# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme

### There seems to be an error when importing the personality models that causes the responses to go quite strange, it may be necessary to train up a model from scratch should this occur. 
To start chatting you need to execute the Tokenisation Function, Ensure Lowercase Function, and Remove Non-alphabetic function code found under 1.2. Then, you may skip straight to loading the Seq2Seq, and loading the word embeddings model. To do this you need to import gensim from the first code cell in 2.1, then run the code cells under 2.1.6, and the variable define in the code cell under 2.1.5-6. Then, execute the code cell in 2.2.1 and the first code cel under 2.2 which imports numpy and tensorflow. From there you are free to move to executing 2.2.2, and then executing 2.2.5 to load the sequence by executing the three code blocks. Now simple execute all cells in section 3, and the chatbot should run.  
To train a whole chatbot from scratch, run the code cells in the order they are presented.

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
# Import all the pydrive things
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Download the things
professional_id = '1WXpmL0tcfJB_nKaI42z-F3kWbOoObdLu'
comic_id = '1PwHiO0iDi1QdAsD87dqwfr9MabGjKGcQ'
friend_id = '1x9X3QeGWgrkkuYcM_Aiw3qb8orBvn-m3'
professional_download = drive.CreateFile({'id':professional_id})
professional_download.GetContentFile('qna_chitchat_the_professional.tsv')
comic_download = drive.CreateFile({'id':comic_id})
comic_download.GetContentFile('qna_chitchat_the_comic.tsv')
friend_download = drive.CreateFile({'id':friend_id})
friend_download.GetContentFile('qna_chitchat_the_friend.tsv')

## 1.2. Preprocess data (Personality chat datasets)

###Steps in preprocessing:
1. Extraction of personality data and storage in dictionary
2. Tokenise Question data
3. Remove stopwords from Questions and Answers
4. Ensure Questions are all lowercase
5. Remove non-alphabetic characters

#### Extraction

In [0]:
import pandas as pd
# Initialise dictionary for the data once preprocessed
friend = pd.read_csv('qna_chitchat_the_friend.tsv', sep='\t')
friend = friend.dropna()
comic = pd.read_csv('qna_chitchat_the_comic.tsv', sep='\t')
comic = comic.dropna()
professional = pd.read_csv('qna_chitchat_the_professional.tsv', sep='\t')
professional = professional.dropna()

personality_data = {
    'friend': friend,
    'comic': comic,
    'professional': professional
}

#### Tokenisation Function
Defines the function to tokenise data (used for both training and chatting)

In [0]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenise(words):
    return word_tokenize(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Stopword Removal Function

In [0]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(words):
    return [w for w in words if w not in stopwords.words()]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Ensure Lowercase Function

In [0]:
def to_lowercase(words):
    return [w.lower() for w in words]

#### Remove Non-alphabetic Characters Function

In [0]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def remove_nonalphabetic(words):
    no_puncts = []
    for w in words:
        w = str(w)
        for p in puncts:
            if p in w:
                w = w.replace(p, '')
        if len(w) > 0:
            no_puncts.append(w)
    return no_puncts

#### Preprocess Personality Data

In [0]:
responses = {'friend': set(), 'professional': set(), 'comic': set()}
resp_dict = {'friend': [], 'professional': [], 'comic': []}
personalities = {'friend':[], 'professional':[], 'comic':[]}
max_lens = {'friend': 0, 'professional': 0, 'comic': 0}

# break up data into question-answer pairs
for key, value in personality_data.items():
    qnas = []
    for index, qna in value.iterrows():
        print(key, ': ', '{:%}'.format(index/value.shape[0]), end='')
        question = qna['Question']
        answer = qna['Answer']
        question = tokenise(question)
        question = to_lowercase(question)
        question = remove_nonalphabetic(question)
        question = list(filter(None, question))
        max_lens[key] = max(max_lens[key], len(question))
        qnas.append((question, answer))
        responses[key].add(answer)
        print('\r', end='')
    print('Basic preprocessing of', key, 'personalty is complete.')
    personalities[key] = list(qnas)


Basic preprocessing of friend personalty is complete.
Basic preprocessing of comic personalty is complete.
Basic preprocessing of professional personalty is complete.


In [0]:
# create dictionaries of responses for each personality
# these need to include the _B_ and _E_
for key, value in responses.items():
    responses[key].add('_B_')
    responses[key].add('_E_')
    responses[key].add('_P_')
    responses[key].add('_U_')
    resp_list = list(responses[key])
    resp_list.sort()
    num_dict = {n: i for i, n in enumerate(resp_list)}
    resp_dict[key] = (resp_list, num_dict)

# 2 - Model Implementation

## 2.1. Word Embeddings

I chose to use FastText as the word embedding model for the ChatBot. This is because FastText is able to deal with words outside of the original corpus, and given the nature of a chatbot, it would be very necessary it is able to understand words that weren't in the training data. For instance, its not uncommon for users to extend the number of letters in words for emphasis, or make words up if they don't know how to properly describe a phenomenon. These are scenarios that can't be covered using a Word2Vec model. 

In [0]:
from gensim.models import FastText

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


### 2.1.1. Download Dataset for Word Embeddings



The dataset chosen was the TED Script dataset mentioned in one of the earlier tutorials. This dataset was both large, and likely to contain a larger variety of word then say, a dataset of movie scripts, owing to the formal, technical style of the TED talk. 
As far as a chat bot is concerned, the more complex, technical, or uncommon words present mightn't be likely to come up - however I felt should they, it would be best to have as accurate a vector as possible. 

In [0]:
# Import all the pydrive things
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')

### 2.1.2. Data Preprocessing for Word Embeddings

The data is preprocessed in the same way as the personality data

In [0]:
import re
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

xml = open('ted_en-20160408.xml', 'r', encoding='UTF8')
xml = etree.parse(xml)
text = ''.join(xml.xpath('//content/text()'))
text = re.sub(r'\([^)]*\)', '', text.lower())
sentences = sent_tokenize(text)
data = []
for sentence in sentences:
    s = tokenise(sentence)
    data.append(remove_nonalphabetic(s))
print(data[:100])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing'], ['consider', 'facit'], ['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them'], ['facit', 'was', 'a', 'fantastic', 'company'], ['they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world'], ['everybody', 'used', 'them'], ['and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along'], ['they', 'continued', 'doing', 'exactly', 'the', 'same'], ['in', 'six', 'months', 'they', 'went', 'from', 'maximum', 'revenue', 'and

### 2.1.3. Build Word Embeddings Model

We're using a CBOW model here. I felt through experimentation during the week 3 lab that the CBOW gave the more acurate sense of similar words when presented with invented words, and words outside of the vocabulary of the original data set.

In [0]:
word_embeddings_model = FastText(
    size=100,
    window=5,
    min_count=5,
    workers=5,
    sg=0,
    word_ngrams=1
)
word_embeddings_model.build_vocab(data)

### 2.1.4. Train Word Embeddings Model

In [0]:
word_embeddings_model.train(sentences=data, total_examples=len(data), epochs=10)

### 2.1.5-6 Local or Cloud model Saving Variable

Swap this variable to True if you wish to not load and save the word embeddings model from the cloud

In [0]:
save_load_embeddings_locally = False

### 2.1.5. Save Word Embeddings Model

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle

def save_embedding(model):
    name = 'word_embeddings_model_BTEM3257.model'
    model.save(name)
    # Authenticate and create the PyDrive client.
    if save_load_embeddings_locally is False:
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        saved_word_embeddings_a = drive.CreateFile({'title': 'word_embeddings_model_BTEM3257.model'})
        saved_word_embeddings_b = drive.CreateFile({'title': 'word_embeddings_model_BTEM3257.model.trainables.vectors_ngrams_lockf.npy'})
        saved_word_embeddings_c = drive.CreateFile({'title': 'word_embeddings_model_BTEM3257.model.wv.vectors_ngrams.npy'})
        
        saved_resp_dict = drive.CreateFile({'title': 'resp_dict'})
        with open('resp_dict.pkl', 'wb') as f:
            pickle.dump(resp_dict, f)


        saved_word_embeddings_a.SetContentFile('./word_embeddings_model_BTEM3257.model')
        saved_word_embeddings_b.SetContentFile('./word_embeddings_model_BTEM3257.model.trainables.vectors_ngrams_lockf.npy')
        saved_word_embeddings_c.SetContentFile('./word_embeddings_model_BTEM3257.model.wv.vectors_ngrams.npy')
        saved_resp_dict.SetContentFile('./resp_dict.pkl')


        saved_word_embeddings_a.Upload()
        saved_word_embeddings_b.Upload()
        saved_word_embeddings_c.Upload()
        saved_resp_dict.Upload()


        print('title: %s, id: %s' % (saved_word_embeddings_a['title'], saved_word_embeddings_a['id']))
        print('title: %s, id: %s' % (saved_word_embeddings_b['title'], saved_word_embeddings_b['id']))
        print('title: %s, id: %s' % (saved_word_embeddings_c['title'], saved_word_embeddings_c['id']))
        print('title: %s, id: %s' % (saved_resp_dict['title'], saved_resp_dict['id']))            

In [0]:
save_embedding(word_embeddings_model)

title: word_embeddings_model_BTEM3257.model, id: 1BE6tQBWx4dHSZVBV_75T-qAMzAOLaLoV
title: word_embeddings_model_BTEM3257.model.trainables.vectors_ngrams_lockf.npy, id: 1rggfUJTNrNz8ph_oC9QXB7gNMfxHYEci
title: word_embeddings_model_BTEM3257.model.wv.vectors_ngrams.npy, id: 1x6o6mMwZ6n2baXFwZySuqaaBAkZIs-hW
title: resp_dict, id: 1o0_R2Z9zftaqBBAVFle7cKBhrF5mD11j


### 2.1.6. Load Word Embeddings Model

In [0]:
word_embeddings_model_a_id = '1BE6tQBWx4dHSZVBV_75T-qAMzAOLaLoV'
word_embeddings_model_b_id = '1rggfUJTNrNz8ph_oC9QXB7gNMfxHYEci'
word_embeddings_model_c_id = '1x6o6mMwZ6n2baXFwZySuqaaBAkZIs-hW'
resp_dict_id = '1o0_R2Z9zftaqBBAVFle7cKBhrF5mD11j'

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle

def load_embeddings():
    # Authenticate and create the PyDrive client.
    if save_load_embeddings_locally is False:
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)
    
        word_embeddings_a = drive.CreateFile({'id': word_embeddings_model_a_id})
        word_embeddings_b = drive.CreateFile({'id': word_embeddings_model_b_id})
        word_embeddings_c = drive.CreateFile({'id': word_embeddings_model_c_id})
        resp_dict_dl = drive.CreateFile({'id': resp_dict_id})

    
        word_embeddings_a.GetContentFile('word_embeddings_model_BTEM3257.model')
        word_embeddings_b.GetContentFile('word_embeddings_model_BTEM3257.model.trainables.vectors_ngrams_lockf.npy')
        word_embeddings_c.GetContentFile('word_embeddings_model_BTEM3257.model.wv.vectors_ngrams.npy')
        resp_dict_dl.GetContentFile('resp_dict.pkl')
        

    model = FastText.load('word_embeddings_model_BTEM3257.model')
    with open('resp_dict.pkl', 'rb') as f:
        r_d = pickle.load(f)
    
    return model, r_d

## 2.2. Seq2Seq model

In [0]:
import numpy as np
import tensorflow as tf

### 2.2.1. Apply/Import Word Embedding Model

In [0]:
word_embeddings, resp_dict = load_embeddings()

### Justification for Hyperparameters
#### Learning rate 0.001:
##### Adam with epsilon 0.1:
Friend - 5002 with a final cost of  0.836100  
Professional - Stopeed at epoch: 5003 with a final cost of  0.932553  
Stopeed at epoch: 5003 with a final cost of  0.864433  

##### Adam with default:
all lower - between 0.001 and 0.004, but more irratic, harder to ensure the lowest loss

##### Adam with epsilon 0.01:
Friend - Stopeed at epoch: 5002 with a final cost of  0.014182  
Professional - Stopeed at epoch: 5002 with a final cost of  0.015218  
Comic - Stopeed at epoch: 5002 with a final cost of  0.014376  

#### Learning Rate 0.01:
##### epsilon 0.1:
Friend - Stopeed at epoch: 5003 with a final cost of  0.011892  
Professional - Stopeed at epoch: 5002 with a final cost of  0.013488  
Comic - Stopeed at epoch: 5002 with a final cost of  0.011853

##### epsilon 0.01
Friend - Stopeed at epoch: 5005 with a final cost of  0.002911  
Professional - Stopeed at epoch: 5002 with a final cost of  0.003078  
Comic - Stopeed at epoch: 5002 with a final cost of  0.007007

#####  epsilon default
Friend - Stopeed at epoch: 5002 with a final cost of  1.052485  
Professional - Stopeed at epoch: 5002 with a final cost of  1.634274  
Stopeed at epoch: 5002 with a final cost of  1.311647

##### epsilon 0.001
Friend - Stopeed at epoch: 5004 with a final cost of  0.422949  
Professional - Stopeed at epoch: 5003 with a final cost of  1.424263  
Comic - Stopeed at epoch: 5003 with a final cost of  1.167939



Given these test results I'm going to take a 0.01 Learning rate and a 0.01 epsilon and increase the epochs from 5000 to 7500, then 10000 to see the restuls
#### 7500 epochs
friend - Stopeed at epoch: 7504 with a final cost of  0.003122  
professional - Stopeed at epoch: 7502 with a final cost of  0.003158  
comic - Stopeed at epoch: 7503 with a final cost of  0.004225

Since it was fairly clear from increasing the epochs that there wasn't a substantial reduction in cost (other than for the comic personality), I decided not to test on 10000 to avoid the risk of overfitting. 

This is how I came up with the minimum epoch of 5000, a learning rate of 0.01 and an 0.01 epsilon value for my models

### 2.2.2. Build Seq2Seq Model

In [0]:
# Global Variables
# Define Model Global Variables    
learning_rate = 0.01
number_hidden = 128

personality_models = {}

number_classes = 105
input_size = 100

max_len = 0

tf.reset_default_graph()
encoder_input = tf.placeholder(tf.float32, [None, None, input_size])
decoder_input = tf.placeholder(tf.float32, [None, None, 105])
targets = tf.placeholder(tf.int64, [None, None])

# Define Encoder Cell
with tf.variable_scope('encode'):
    encoder_cell = tf.nn.rnn_cell.BasicRNNCell(number_hidden)
    encoder_cell = tf.nn.rnn_cell.DropoutWrapper(encoder_cell, output_keep_prob=0.5)    
    outputs, encoder_states = tf.nn.dynamic_rnn(encoder_cell, encoder_input, dtype=tf.float32)
        
# Define Decoder Cell
with tf.variable_scope('decode'):
    decoder_cell = tf.nn.rnn_cell.BasicRNNCell(number_hidden)
    decoder_cell = tf.nn.rnn_cell.DropoutWrapper(decoder_cell, output_keep_prob=0.5)
    outputs, decoder_states = tf.nn.dynamic_rnn(decoder_cell, decoder_input, initial_state=encoder_states, dtype=tf.float32)

# Define Model
model = tf.layers.dense(outputs, number_classes, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimiser = tf.train.AdamOptimizer(learning_rate, epsilon=0.01).minimize(cost)
   
session = tf.Session()
session.run(tf.global_variables_initializer())

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
# Function Definitions
def get_q_vector(question):
    # need to pad this so they all have the same length
    difference = max_len - len(question)
    for _ in range(difference):
        question.append('_P_')
    vectors = []
    for token in question:
        vectors.append(word_embeddings[token])
    return vectors
    

def get_a_vector(answer, r_i):
    i = r_i[answer]
    return i

def make_batch(sequence_data, r_i):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in sequence_data:
        input_data = get_q_vector(seq[0])
        output_data = [get_a_vector('_B_', r_i), get_a_vector(seq[1], r_i)]
        target_data = [get_a_vector(seq[1], r_i), get_a_vector('_E_', r_i)]
        
        input_batch.append(input_data)
        output_batch.append(np.eye(105)[output_data])
        target_batch.append(target_data)
    
    return input_batch, output_batch, target_batch
    

### 2.2.3. Train Seq2Seq Model

In [0]:
total_epochs = 5000
for key, value in personalities.items():
    max_len = max_lens[key]
    response_indexes = resp_dict[key][1]
        
    input_batch, output_batch, target_batch = make_batch(personalities[key], response_indexes)

        
    print('Now training ', key, ' model')
    session.run(tf.global_variables_initializer())
    
    curr_epoch = 0
    prev_loss = 0
    curr_loss = -1
    
    while True:
        _, curr_loss = session.run([optimiser, cost],
                             feed_dict={
                                 encoder_input: input_batch,
                                 decoder_input: output_batch,
                                 targets: target_batch
                             })
        if curr_epoch > total_epochs:
            if curr_loss > prev_loss:
                print('Stopeed at epoch: %04d' % (curr_epoch+1), 'with a final cost of ', '{:.6f}'.format(curr_loss))
                break
        if curr_epoch % 100 == 0:
            print('Epoch:', '%04d' % (curr_epoch + 1),
            'cost =', '{:.6f}'.format(curr_loss))
        prev_loss = curr_loss
        curr_epoch += 1
            
    saver = tf.train.Saver()
    save_path =  './' + key
    saver.save(session, save_path)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Now training  friend  model
Epoch: 0001 cost = 4.731266
Epoch: 0101 cost = 2.246108
Epoch: 0201 cost = 1.520328
Epoch: 0301 cost = 0.510572
Epoch: 0401 cost = 0.136607
Epoch: 0501 cost = 0.062369
Epoch: 0601 cost = 0.035651
Epoch: 0701 cost = 0.024743
Epoch: 0801 cost = 0.017703
Epoch: 0901 cost = 0.015311
Epoch: 1001 cost = 0.012084
Epoch: 1101 cost = 0.011020
Epoch: 1201 cost = 0.011098
Epoch: 1301 cost = 0.008911
Epoch: 1401 cost = 0.008126
Epoch: 1501 cost = 0.006996
Epoch: 1601 cost = 0.007357
Epoch: 1701 cost = 0.006110
Epoch: 1801 cost = 0.005293
Epoch: 1901 cost = 0.005254
Epoch: 2001 cost = 0.005044
Epoch: 2101 cost = 0.004210
Epoch: 2201 cost = 0.004328
Epoch: 2301 cost = 0.005125
Epoch: 2401 cost = 0.004316
Epoch: 2501 cost = 0.005902
Epoch: 2601 cost = 0.004451
Epoch: 2701 cost = 0.005667
Epoch: 2801 cost = 0.004035
Epoch: 2901 cost = 0.003969
Epoch: 3001 cost = 0.004172
Epoch: 3101 cost = 0.003924
Epoch: 3201 cost = 0.003825
Epoch: 3301 cost = 0.003687
Epoch: 3401 cost = 0

### 2.2.4. Save Seq2Seq Model

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def save_sequence():
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    
    saved_checkpoint = drive.CreateFile({'title':'checkpoint'})
    saved_checkpoint.SetContentFile('./checkpoint')
    saved_checkpoint.Upload()
    print('title: %s, id: %s' % (saved_checkpoint['title'], saved_checkpoint['id']))
    
    for p in ['friend', 'comic', 'professional']:
        saved_a = drive.CreateFile({'title': p+'.index'})
        saved_b = drive.CreateFile({'title': p+'.meta'})    
        saved_c = drive.CreateFile({'title': p+'.data-00000-of-00001'})    
    
        saved_a.SetContentFile('./'+p+'.index')
        saved_b.SetContentFile('./'+p+'.meta')
        saved_c.SetContentFile('./'+p+'.data-00000-of-00001')    
    
        saved_a.Upload()
        saved_b.Upload()
        saved_c.Upload()


        print('title: %s, id: %s' % (saved_a['title'], saved_a['id']))
        print('title: %s, id: %s' % (saved_b['title'], saved_b['id']))
        print('title: %s, id: %s' % (saved_c['title'], saved_c['id']))    

In [0]:
save_sequence()

title: checkpoint, id: 1VCSfWQyIlmOgkokkPqtEe-JzQ6LjyPr3
title: friend.index, id: 1Z5_GsJABoejrYOVCHi3T8_KRpbfFM41c
title: friend.meta, id: 18SyVsGalR0ApqSLVQVnVPd2QDpuA_v_D
title: friend.data-00000-of-00001, id: 1H162ZArHs6mkaIYw0SbSIQryeWegO4lO
title: comic.index, id: 17XpnG9vYUI067GW4pkZfiCanE_0sSVTU
title: comic.meta, id: 1RJSR-qgxITc13UpacuObkTuMp0Vkc902
title: comic.data-00000-of-00001, id: 1Uc7Jl1uOQkTtLNBBWZq5i46I5b-2B3NW
title: professional.index, id: 1mh-aPFdWBYuTDB8DVEd-1a20L6FBuisb
title: professional.meta, id: 1fFBplK8F8tdHexgrU9-lblYt3z_jHGXY
title: professional.data-00000-of-00001, id: 1JlDnnKTymjzuFmgMjavHcsfrxiYAvTXu


### 2.2.5. Load Seq2Seq Model

In [0]:
check = '1VCSfWQyIlmOgkokkPqtEe-JzQ6LjyPr3'
a = ['1Z5_GsJABoejrYOVCHi3T8_KRpbfFM41c', '17XpnG9vYUI067GW4pkZfiCanE_0sSVTU', '1mh-aPFdWBYuTDB8DVEd-1a20L6FBuisb']
b = ['18SyVsGalR0ApqSLVQVnVPd2QDpuA_v_D', '1RJSR-qgxITc13UpacuObkTuMp0Vkc902', '1fFBplK8F8tdHexgrU9-lblYt3z_jHGXY']
c = ['1H162ZArHs6mkaIYw0SbSIQryeWegO4lO', '1Uc7Jl1uOQkTtLNBBWZq5i46I5b-2B3NW', '1JlDnnKTymjzuFmgMjavHcsfrxiYAvTXu']

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def load_sequence():
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    
    download_check = drive.CreateFile({'id': check})
    download_check.GetContentFile('checkpoint')
    
    i = 0
    for p in ['friend', 'comic', 'professional']:
        download_a = drive.CreateFile({'id': a[i]})
        download_b = drive.CreateFile({'id': b[i]})
        download_c = drive.CreateFile({'id': c[i]})
    
        download_a.GetContentFile(p+'.index')
        download_b.GetContentFile(p+'.meta')    
        download_c.GetContentFile(p+'.data-00000-of-00001')   
        i += 1

In [0]:
load_sequence()

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
def begin_chat():
    string = "Bot: Hi!\nBot: I'm the NLP ChatBot, you can change my personalities by saying 'Change Personality to' and then your personality of choice.\nBot: There are three to choose from: Professional, Comic, and Friend.\nBot: You can stop chatting by typing 'Goodbye,' and I can save a log of our chat if you tell me 'Save Chat'"
    log.append(string)
    print(string)
    return 'friend'

## 3.2. Change Personality

To change the personality simply type the message "Change personality to " followed by the personality you wish to switch to 

In [0]:
def change_personality(p):
    if p.lower() not in ['friend', 'comic', 'professional']:
        print("Sorry, I don't have a personality by that name. My personalities are 'friend', 'comic', and 'professional'")
        return False
    else:
        return True

## 3.3. Save chat log

In [0]:
import datetime

def save_chat():
    chat_log = open('Chatlog'+str(datetime.datetime.now())[:19]+'.txt', 'w')
    for l in log:
        chat_log.write(l+'\n')
    chat_log.close()

## 3.4. End chatting

In [0]:
def end_chat():
    log.append('Bot: Goodbye!')
    print('Bot: Goodbye!')

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

To train the chatbot again you simply need to follow down the code cells from section 1 down to section 2.2.3. Theres no need to save or load the seq2seq model since these functions simply save it to the google drive. You can swap the variable "local" in the word embedding load and save functions to skip uploading them to the google drive.

### 3.5.2. Execute program - chatting mode

Run the afforementioned functions in 3.1 to 3.4 to define the needed functions, then execute the following code cells in order

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
def answer(question):
    question = tokenise(question)
    question = to_lowercase(question)
    question = remove_nonalphabetic(question)
    question = list(filter(None, question))
    
    sequence_data = [question, '_U_']
    input_batch, output_batch, target_batch = make_batch([sequence_data], response_indexes)
    
    
    session = tf.Session()
    saver = tf.train.Saver()
    saver.restore(session, './'+personality)
    prediction = tf.argmax(model, 2)
    result = session.run(prediction, feed_dict={
                            encoder_input: input_batch,
                            decoder_input: output_batch,
                            targets: target_batch
                        })
    return responses[result[0][0]]

In [0]:
chatting = True
log = []
personality = begin_chat()

responses = resp_dict[personality][0]
response_indexes = resp_dict[personality][1]

while(chatting):
    message = input()
    log.append('You:' + message)
    message = message.lower()
    if message.startswith('change personality to'):
        p = message.split()[-1]
        if change_personality(p):
            personality = p
            responses = resp_dict[personality][0]
            response_indexes = resp_dict[personality][1]
    elif message == 'goodbye':
        end_chat()
        chatting = False
    elif message == 'save chat':
        save_chat()
    elif len(message) == 0:
        bot_answer = "You didn't say anything..."
        log.append('Bot:' + bot_answer)
        print("Bot:", bot_answer)
    else:
        bot_answer = answer(message)
        log.append('Bot:' + bot_answer)
        print("Bot:", bot_answer)


Bot: Hi!
Bot: I'm the NLP ChatBot, you can change my personalities by saying 'Change Personality to' and then your personality of choice.
Bot: There are three to choose from: Professional, Comic, and Friend.
Bot: You can stop chatting by typing 'Goodbye,' and I can save a log of our chat if you tell me 'Save Chat'
Hi
Bot: Hi!
How are you?
Bot: I'm doing great, thanks for asking!
Do you have any parents?
Bot: I haven't met any other bots, but I bet we'd get along.


KeyboardInterrupt: ignored

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon